# Task execution by a comittee of AI agents

In the last example we used two agents to simplify and execute a well known ML example. In this task we undertake a slightly more challenging task of creating and training a Deep Learning model for image classification of flowers.

The group of agents consists of:

1. Planner: to break down a complex task into sub-tasks
2. Engineer: Who will take each task and write code for it
3. Executor: Will execute code and provide feedback
4. Writer: To write a post about the process (see next page)
5. Admin: This is the human user who is asked occasional questions to steer the project. 

In this task very little input from the Admin was needed and the model performed quite well for something which was run for a short time as a test. 

The interesting bit in this trial was we asked for the AI training code to be executed external to the chat with the agents and the agents then picked up the task after the model was trained.

Next: Checkout the blog written about the process: [Building an Image Classifier for Flower Recognition: A Journey with TensorFlow and Keras](https://ai.aneeshsathe.com/Group%20Task%20Decomp/Group_chat.html)

![](work_dir/fl1.jpg) ![](work_dir/fl2.jpg) ![](work_dir/fl3.jpg) ![](work_dir/confusion_matrix.png)

## Environment Setup

In [1]:
import os
from datetime import datetime
from typing import Callable, Dict, Literal, Optional, Union

from typing_extensions import Annotated

from autogen import (
    AssistantAgent,
    GroupChat,
    GroupChatManager,
    UserProxyAgent,
    config_list_from_json,
)
from autogen.cache import Cache
from autogen.coding import LocalCommandLineCodeExecutor

work_dir = "work_dir"

In [2]:
config_list_path = "../json_configs/OAI_CONFIG_LIST.json"
config_list = config_list_from_json(env_or_file=config_list_path)

## Task definition

In [3]:
dataset_url = os.getcwd()+"/work_dir/flower_data/flower_photos"
task = (
    f"Build an image classifier for the flower classification dataset available here: {dataset_url}. Do exploratory data analysis to get a sense of the data. Use keras and tensorflow libraries for the task. Show the performance of the model in the standard ways. Save plots and figures with consecutive numbers in the filename. Finally write a short blog post about the process used to develop this model and showing the peroformance of the model. Use the task_planner function to break down task into substeps. For the training and validation step, provide a separate .py file which saves the model, output and statistics that i can run separately and then you can pick up the results from the saved files."
)

## Setup Comittee of Agents and Execute Task

In [5]:
user_proxy = UserProxyAgent(
    name="Admin",
    system_message="A human admin. Give the task, and send instructions to writer to refine the blog post.",
    code_execution_config=False,
)

planner = AssistantAgent(
    name="Planner",
    system_message="""Planner. Given a task, please determine what information is needed to complete the task.
Please note that the information will all be retrieved using Python code. Please only suggest information that can be retrieved using Python code.
""",
    llm_config={"config_list": config_list, "cache_seed": None},
)

engineer = AssistantAgent(
    name="Engineer",
    llm_config={"config_list": config_list, "cache_seed": None},
    system_message="""Engineer. You write python/bash to retrieve relevant information. Wrap the code in a code block that specifies the script type. The user can't modify your code. So do not suggest incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by the executor.
Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. Check the execution result returned by the executor.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
""",
)

writer = AssistantAgent(
    name="Writer",
    llm_config={"config_list": config_list, "cache_seed": None},
    system_message="""Writer. Please write blogs in markdown format (with relevant titles) and put the content in pseudo ```md``` code block. You will write it for a task based on previous chat history. Don't write any code.""",
)

# os.makedirs("paper", exist_ok=True)
code_executor = UserProxyAgent(
    name="Executor",
    system_message="Executor. Execute the code written by the engineer and report the result.",
    description="Executor should always be called after the engineer has written code to be executed.",
    human_input_mode="ALWAYS",
    code_execution_config={
        "last_n_messages": 3,
        "executor": LocalCommandLineCodeExecutor(work_dir=work_dir),
    },
)

groupchat = GroupChat(
    agents=[user_proxy, engineer, code_executor, writer, planner],
    messages=[],
    max_round=50,
    speaker_selection_method="auto",
)
manager = GroupChatManager(groupchat=groupchat, llm_config={"config_list": config_list, "cache_seed": None})

# Use Cache.disk to cache LLM responses. Change cache_seed for different responses.
with Cache.disk(cache_seed=41) as cache:
    chat_history = user_proxy.initiate_chat(
        manager,
        message=task,
        cache=cache,
    )

Admin (to chat_manager):

Build an image classifier for the flower classification dataset available here: /Users/aneeshsathe/analysis_work/autogen/Group Chat Task Decomposition/work_dir/flower_data/flower_photos. Do exploratory data analysis to get a sense of the data. Use keras and tensorflow libraries for the task. Show the performance of the model in the standard ways. Save plots and figures with consecutive numbers in the filename. Finally write a short blog post about the process used to develop this model and showing the peroformance of the model. Use the task_planner function to break down task into substeps. For the training and validation step, provide a separate .py file which saves the model, output and statistics that i can run separately and then you can pick up the results from the saved files.

--------------------------------------------------------------------------------
Planner (to chat_manager):

To accomplish the task of building an image classifier for the flower 

In [ ]:
# Training script

import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths
base_dir = '/Users/aneeshsathe/analysis_work/autogen/Group Chat Task Decomposition/work_dir/flower_data/flower_photos'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Image dimensions and batch size
img_height, img_width = 150, 150
batch_size = 32

# Create ImageDataGenerators for training and validation
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.2,
                                   height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(img_height, img_width),
                                                    batch_size=batch_size, class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(validation_dir, target_size=(img_height, img_width),
                                                              batch_size=batch_size, class_mode='categorical')

# Model architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')  # Adjust the final layer according to number of classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model Training
history = model.fit(train_generator, epochs=30, validation_data=validation_generator, verbose=1)

# Save the model and training history
model.save('flower_classifier_model.h5')
with open('model_history.txt', 'w') as f:
    f.write(str(history.history))

print("Model trained and saved successfully.")

![](work_dir/training.png)

In [ ]:
# Eval script

import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Load the model
model_path = '/Users/aneeshsathe/analysis_work/autogen/Group Chat Task Decomposition/work_dir/flower_classifier_model.h5'
model = tf.keras.models.load_model(model_path)

# Prepare test data
base_dir = '/Users/aneeshsathe/analysis_work/autogen/Group Chat Task Decomposition/work_dir/flower_data/flower_photos'
test_dir = os.path.join(base_dir, 'test')
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(test_dir, target_size=(150, 150),
                                                  batch_size=32, class_mode='categorical', shuffle=False)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator, verbose=1)
print(f"Test accuracy: {test_acc*100:.2f}%, Test loss: {test_loss}")

# Predictions for confusion matrix
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

# Confusion Matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)
plt.figure(figsize=(8, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.savefig('confusion_matrix.png')
plt.show()

# Classification report
print(classification_report(true_classes, predicted_classes, target_names=class_labels))

![](work_dir/confusion_matrix.png)